**Exercise 1:**

$ \text{max}\  0.85x_1 + 3.75x_2 -8.75x_3 - 3.45x_4 - 22.38x_5 + 20x_6 \\ \text{subjected to}\ -0.7x_1 -3.9x_2 -x_3+ x_5 \leq 14.9, \\ x_1 - x_4 -x_6 \leq 0.9, \\ -3x_2 - 2x_3 -x_6 \leq -51.7, \\ 4.8x_4 + 9.12x_5 + 7.2x_6 \leq 75, \\ -0.8x_1 - 1.5x_2 + 8x_3 -2x_4 + x_5 \leq -27, \\ x_i \geq 0 \text{ }\ \forall i \in \{1,2,3,4,5,6 \} $

In [1]:
!pip install -q pyomo
from pyomo.environ import *
import numpy as np

     |████████████████████████████████| 9.1 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 


In [2]:
coef = np.loadtxt('lab4_ex1_coef.txt',delimiter=',')

In [3]:
print(coef)

[[  0.85   3.75  -8.75  -3.45 -22.38  20.     0.  ]
 [ -0.7   -3.9   -1.     0.     1.     0.    14.9 ]
 [  1.     0.     0.    -1.     0.    -1.     0.9 ]
 [  0.    -3.    -2.     0.     0.    -1.   -51.7 ]
 [  0.     0.     0.     4.8    9.12   7.2   75.  ]
 [ -0.8   -1.5    8.    -2.     1.     0.   -27.  ]]


In [4]:
model = ConcreteModel()

In [5]:
N = coef.shape[1]-1
M = coef.shape[0]-1

In [6]:
obj_coef = coef[0,:-1]
constr_coef = coef[1:,:-1]
constr_rhs = coef[1:,-1]

In [7]:
lower_bound = 0
upper_bound = np.inf

In [8]:
col_indices = np.arange(N)

In [9]:
model.x = Var(col_indices)


In [10]:
row_indices = np.arange(M)


In [11]:
model.constraints = ConstraintList()

In [12]:
for i in row_indices:
  model.constraints.add(summation(constr_coef[i],model.x) <= constr_rhs[i])

In [13]:
for j in col_indices:
  model.x[j].setlb(lower_bound)
  model.x[j].setub(upper_bound)

In [14]:
model.objective = Objective(expr = summation(obj_coef, model.x), sense = maximize)

In [15]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.85*x[0] + 3.75*x[1] - 8.75*x[2

In [16]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148489 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [17]:
opt_cbc = SolverFactory('cbc')

In [18]:
model.objective.set_sense(maximize)

In [19]:
result = opt_cbc.solve(model)

    model.name="unknown";
      - termination condition: unbounded
      - message from solver: <undefined>


In [20]:
print(result)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: None
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 6
  Sense: maximize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: unbounded
  Termination message: Model was proven to be unbounded.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.015654802322387695



In [21]:
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Solver status: warning
Solver termination condition: unbounded


Termination message is : Model was proven to be unbounded

It means that the solution of this model which satisfy all the given constraint is unbounded so it is not possible to find the optimal vlaue of the objective function 

In [22]:
model.objective.set_sense(minimize)

In [23]:
result = opt_cbc.solve(model)

In [24]:
print(result)


Problem: 
- Name: unknown
  Lower bound: -97.02184211
  Upper bound: -97.02184211
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 6
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.030386924743652344
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [25]:
print("Objective= ",model.objective())
for i in col_indices:
  print("model.x[i]",model.x[i].value)

print('\nConstraints')
model.constraints.display()


Objective=  -97.02184239599997
model.x[i] 0.9
model.x[i] 23.002456
model.x[i] 0.0
model.x[i] 0.0
model.x[i] 8.2236842
model.x[i] 0.0

Constraints
constraints : Size=5
    Key : Lower : Body                : Upper
      1 :  None :         -82.1158942 :  14.9
      2 :  None :                 0.9 :   0.9
      3 :  None :          -69.007368 : -51.7
      4 :  None :   74.99999990399999 :  75.0
      5 :  None : -26.999999799999998 : -27.0


remedy is to change the sense of the objective function minimizaton

So the objective is now changed to minimization and it gives an optimal value 